# MOCHA scATACseq tracks visualization
1. [Inputs](#Inputs)
2. [Imports](#Imports)
3. [JupyterDash Setup](#JupyterDash-Set-up)
4. [Data Import](#Data-Import)
5. [App](#App)

## Inputs

Provide the directory containing all of the exported files' subdirectories.

In [9]:
data_dir = "/home/jupyter/data"

__Ensure all provided subdirectories are empty other than the files you wish to visualize.__

Set the directory under which coverage .bw files are located.
All coverage files must be named in the format `CELLTYPE__SAMPLEID.bw` (`.bigwig` will not be recognized).

In [3]:
bw_dir = "./data/sample_specific_coverage/"

Set the directory under which tile .bigBed files are located.

In [4]:
tiles_dir = "./data/tiles_cellpop/"

In [5]:
differential_tiles_dir = "./data/tiles_differential/"

Set the directory under which motif bigBed files are located, and the name of the motifset.

We assume the motifsets are named in the pattern `CELLTYPE__MOTIFSETNAME.bigBed`. 

CELLTYPE should be "ALL" if the .bigBed should display for all cell types.

In [6]:
motifs_dir = "./data/motifs/"
motifset = "CISBP" # "Vierstra" etc

Set an order for your cell populations - this dictates the order tracks will be displayed in the app.

In [ ]:
ct_order = [
    "t_cd4_naive",
    "t_cd4_cm",
    "t_cd4_em",
    "t_cd4_treg_naive",
    "t_cd4_treg_memory",
    "t_cd8_naive",
    "t_cd8_cm",
    "t_cd8_em",
    "t_cd8_temra",
    "t_cd8_mait",
]

## Imports

In [2]:
# these libraries are all utilized to create the dash app
import dash
from dash.dependencies import Input, Output
import dash_bio as dashbio
from dash import html, dcc
from flask import send_from_directory

In [3]:
import pandas as pd
import re
import os
import glob
import time

In [4]:
! python --version

Python 3.7.12


In [5]:
dash.__version__ # 2.10.2 or newer

'2.10.2'

In [6]:
dashbio.__version__ # 0.9.0 works. 1.0.2 may not work on Python 3.7

'0.9.0'

If these are not installed, set below cells to "Code" and run:

! pip install dash

! pip install dash-bio==0.9.0

! pip install jupyterlab "ipywidgets>=7.5”
! jupyter labextension install jupyterlab-plotly@4.14.3

## JupyterDash Set-up
The following 2 code-blocks are needed if you are trying to develop a dash-app in a HISE IDE.

NOTE: Code in this block ultimately shouldn't be included when deploying your dash app to HISE's Collab Space

In [7]:
from jupyter_dash.comms import _send_jupyter_config_comm_request  # jupyter-only

_send_jupyter_config_comm_request()  # jupyter-only

In [8]:
time.sleep(10)

In [10]:
from jupyter_dash import JupyterDash  # jupyter-only

JupyterDash.infer_jupyter_proxy_config()  # jupyter-only

## Data Import

### BigWig Files

In [17]:
bw_fp = glob.glob(os.path.join(bw_dir, "*.bw"))
bw_fp = [os.path.basename(x) for x in bw_fp]

### BED Files
For including tiles tracks - skip if not using tiles or celltype-peak motifs.

In [ ]:
tiles_fp = glob.glob(os.path.join(tiles_dir, "*.bigBed"))
tiles_fp = [os.path.basename(x) for x in tiles_fp]

difftiles_fp = glob.glob(os.path.join(differential_tiles_dir, "*.bigBed"))
difftiles_fp = [os.path.basename(x) for x in difftiles_fp]

In [ ]:
motifs_fp = glob.glob(os.path.join(motifs_dir, f"*__{motifset}.bigBed"))
motifs_fp = [os.path.basename(x) for x in motifs_fp]
motifs_fp

### Format Track Metadata

In [13]:
ct_order = ct_order + ["ALL"]
track_class_order = ["coverage", "sample_tiles", "differential_tiles", "tiles", "motifs"]

In [26]:
subdirectory = "igv/"
fp = bw_fp
track_class = "coverage"
metadf = pd.DataFrame(
    {
        "filename": fp,
        "path": [subdirectory + x for x in bw_fp],
        "celltype": pd.Categorical(
            [re.match("(.*)__.*", x).group(1) for x in fp], categories=ct_order
        ),
        "sample_id": [re.match(".*__(.*)", x).group(1) for x in bw_fp],
        "track_class": "coverage",
    }
)
metadf["label"] = metadf.apply(
    lambda row: row["sample_id"] + ": " + row["celltype"], axis=1
)
metadf.sort_values(["celltype"], inplace=True)

In [32]:
motifs_df = pd.DataFrame(
    {
        "filename": motifs_fp,
        "path": ["igv/motifs/" + x for x in motifs_fp],
        "celltype": pd.Categorical(
            [re.match("(.*)__.*", x).group(1) for x in motifs_fp],
            categories=ct_order,
        ),
        "motifset": motifset,
        "track_class": "motifs",
    }
)
motifs_df["label"] = motifs_df.apply(
    lambda row: row["motifset"] + ": " + row["celltype"], axis=1
)
motifs_df.sort_values(["celltype"], inplace=True)

,filename,path,celltype,sample_id,treat_time,treatment,timepoint,motifset,track_class,label
7,t_cd4_naive__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd4_naive__CISBP__ArchRMotifset.b...,t_cd4_naive,None,None,None,None,CISBP,motifs,CISBP: t_cd4_naive
0,t_cd4_cm__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd4_cm__CISBP__ArchRMotifset.bigBed,t_cd4_cm,None,None,None,None,CISBP,motifs,CISBP: t_cd4_cm
6,t_cd4_em__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd4_em__CISBP__ArchRMotifset.bigBed,t_cd4_em,None,None,None,None,CISBP,motifs,CISBP: t_cd4_em
2,t_cd4_treg_naive__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd4_treg_naive__CISBP__ArchRMotif...,t_cd4_treg_naive,None,None,None,None,CISBP,motifs,CISBP: t_cd4_treg_naive
9,t_cd4_treg_memory__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd4_treg_memory__CISBP__ArchRMoti...,t_cd4_treg_memory,None,None,None,None,CISBP,motifs,CISBP: t_cd4_treg_memory
8,t_cd8_naive__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd8_naive__CISBP__ArchRMotifset.b...,t_cd8_naive,None,None,None,None,CISBP,motifs,CISBP: t_cd8_naive
1,t_cd8_cm__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd8_cm__CISBP__ArchRMotifset.bigBed,t_cd8_cm,None,None,None,None,CISBP,motifs,CISBP: t_cd8_cm
4,t_cd8_em__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd8_em__CISBP__ArchRMotifset.bigBed,t_cd8_em,None,None,None,None,CISBP,motifs,CISBP: t_cd8_em
5,t_cd8_temra__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd8_temra__CISBP__ArchRMotifset.b...,t_cd8_temra,None,None,None,None,CISBP,motifs,CISBP: t_cd8_temra
3,t_cd8_mait__CISBP__ArchRMotifset.bigBed,igv/motifs/t_cd8_mait__CISBP__ArchRMotifset.bi...,t_cd8_mait,None,None,None,None,CISBP,motifs,CISBP: t_cd8_mait


In [ ]:
subdirectory <- "igv/tiles_samplespecific/"
fp = tiles_fp
track_class = "tiles"
tiles_df = pd.DataFrame(
    {
        "filename": tiles_fp,
        "path": [subdirectory + x for x in fp],
        "celltype": None,
        "sample_id": tiles_fp,
        "treat_time": None,
        "treatment": None,
        "timepoint": None,
        "track_class": track_class,
    }
)
tiles_df["label"] = tiles_df.apply(
    lambda row: "Open tiles: " + row["celltype"], axis=1
)
tiles_df.sort_values(["celltype"], inplace=True)
tiles_df

In [ ]:
subdirectory = "igv/tiles_differential/"
fp = difftiles_fp
track_class = "differential_tiles"
differentials_df = pd.DataFrame(
    {
        "filename": fp,
        "path": [subdirectory + x for x in fp],
        "celltype": None,
        "sample_id": None,
        "treat_time": None,
        "treatment": None,
        "timepoint": None,
        "track_class": track_class,
    }
)

differentials_df["label"] = differentials_df.apply(
    lambda row: "Difftiles: " + row["filename"] + ": ", axis=1
)

## App

In [33]:
app = JupyterDash(
    __name__,
    external_stylesheets=[
        "https://storage.googleapis.com/aifi-static-assets/hise-style.css"
    ],
)
server = app.server
app.config.suppress_callback_exceptions = True

colors = {
    "--bs-blue": "#0d6efd",
    "--bs-indigo": "#6610f2",
    "--bs-purple": "#6f42c1",
    "--bs-pink": "#d63384",
    "--bs-red": "#dc3545",
    "--bs-orange": "#fd7e14",
    "--bs-yellow": "#ffc107",
    "--bs-green": "#198754",
    "--bs-teal": "#20c997",
    "--bs-cyan": "#0dcaf0",
    "--bs-white": "#FFFFFF",
    "--bs-gray": "#6c757d",
    "--bs-gray-dark": "#343a40",
    "--bs-gray-100": "#f8f9fa",
    "--bs-gray-200": "#e9ecef",
    "--bs-gray-300": "#dee2e6",
    "--bs-gray-400": "#ced4da",
    "--bs-gray-500": "#adb5bd",
    "--bs-gray-600": "#6c757d",
    "--bs-gray-700": "#495057",
    "--bs-gray-800": "#343a40",
    "--bs-gray-900": "#212529",
    "--bs-hise-blue-1": "#003056",
    "--bs-hise-blue-2": "#325876",
    "--bs-hise-blue-3": "#5286b0",
    "--bs-hise-blue-4": "#71899c",
    "--bs-hise-blue-5": "#b4c3cf",
    "--bs-hise-blue-6": "#d9e0e6",
    "--bs-hise-teal-1": "#33B0C8",
    "--bs-hise-teal-2": "#76CFE0",
    "--bs-hise-teal-3": "#DEF2F6",
    "--bs-hise-grey-1": "#272D3B",
    "--bs-hise-grey-2": "#3E3E3E",
    "--bs-hise-grey-3": "#616161",
    "--bs-hise-grey-4": "#707070",
    "--bs-hise-grey-5": "#ECEDEE",
    "--bs-hise-grey-6": "#FBFBFB",
    "--bs-hise-green-1": "#E3EADA",
    "--bs-hise-green-2": "#A0C572",
    "--bs-hise-green-3": "#94BC62",
    "--bs-hise-green-4": "#4AD991",
    "--bs-aifi-new-001": "#003057",
    "--bs-aifi-new-002": "#5da7e5",
    "--bs-aifi-new-003": "#74A03E",
    "--bs-aifi-new-004": "#f4a261",
    "--bs-aifi-new-005": "#e76f51",
    "--bs-aifi-new-006": "#FFFFD0",
}


# map a url path to a specific rule
# @server.route("{}igv/<path:fname>".format(app.config.url_base_pathname))
@server.route("/igv/<path:fname>")
def send_file(fname):
    return send_from_directory(data_dir, fname)


In [34]:
# Select celltype
@app.callback(
    Output("default-igv-container", "children"),
    Input("default-igv-container", "children"),
    Input("select_celltype", "value"),
    Input("select_treat", "value"),
    Input("select_time", "value"),
    Input("select_sort", "value"),
    Input("show-tiles", "value"),
    Input("genome-igv", "locus"),
    Input("stored-locus", "data"),
)
def update_celltype_tracks(
    in_container,
    celltypes,
    treat,
    tp,
    select_sort,
    show_tiles,
    locus,
    stored_locus_json,
):

    if stored_locus_json:
        stored_locus = json.loads(stored_locus_json)
        print(f"Stored Locus: {stored_locus}")
    
    selected_track_types = show_tiles
    print(f"Locus: {locus}")
    print(f"selected_track_types: {selected_track_types}")

    show_tiles = 'Population-specific tiles' in selected_track_types
    show_difftiles = 'Differential tiles' in selected_track_types
    show_motifs = 'Motif Annotations' in selected_track_types

    if celltypes is None:
        print("nocelltypes selected")
        return in_container
    else:
        print("celltypes: ", celltypes)
        
        # Display tracks that apply to ALL celltypes
        celltypes = celltypes + ["ALL"]
        
        # filter and sort metadata
        ct_meta = metadf[metadf["celltype"].isin(celltypes)]
        motif_meta = motifs_df[motifs_df["celltype"].isin(celltypes)]
        
        tiles_meta = tiles_df[tiles_df["celltype"].isin(celltypes)]
        differentials_meta = differentials_df[
            differentials_df["celltype"].isin(celltypes)
        ]

        if show_difftiles:
            ct_meta = pd.concat([ct_meta, differentials_meta])

        if select_sort is not None:
            ct_meta.sort_values(select_sort, inplace=True)
            motif_meta.sort_values(select_sort, inplace=True)
            # tiles_meta.sort_values(select_sort, inplace=True)

        ct_meta.reset_index(inplace=True)
        motif_meta.reset_index(inplace=True)
        tiles_meta.reset_index(inplace=True)

        # print("tiles: ", tiles, ", difftiles: ", difftiles, ", motifs: ", motifs)
        print("Compiling tracks")
        print("motif_meta: ", motif_meta.shape)
        print("ct_meta: ", ct_meta.shape)
        # Compile
        out_tracks = []
        print("compiling coverage")
        for irow in range(ct_meta.shape[0]):
            # print (irow)
            print(ct_meta["label"][irow])
            if ct_meta["track_class"][irow] == "coverage":
                out_tracks.append(
                    dict(
                        type="wig",
                        name=ct_meta["label"][irow],
                        url=ct_meta["path"][irow],
                        color=ct_meta["colors_treat_time"][irow],
                        autoscaleGroup=ct_meta["track_class"][irow],
                    )
                )
            elif ct_meta["track_class"][irow] == "differential_tiles":
                out_tracks.append(
                    dict(
                        type="annotation",
                        name=ct_meta["label"][irow],
                        displayMode="squished",
                        url=ct_meta["path"][irow],
                        height=35,
                        color="#ff5c5c",
                    )
                )

        if show_tiles:
            print("compiling celltype tiles")
            for irow in range(tiles_meta.shape[0]):
                # print(tiles_meta["label"][irow])
                print(tiles_meta["path"][irow])
                out_tracks.append(
                    dict(
                        type="annotation",
                        name=tiles_meta["label"][irow],
                        displayMode="squished",
                        url=tiles_meta["path"][irow],
                        height=45,
                        color="#ff5c5c",
                    )
                )

        if show_motifs:
            print("compiling celltype motifs")
            for irow in range(motif_meta.shape[0]):
                out_tracks.append(
                    dict(
                        type="annotation",
                        name=motif_meta["label"][irow],
                        displayMode="expanded",
                        url=motif_meta["path"][irow],
                        height=140,
                        color="#ff5c5c",
                        altColor="#ff8c5c"
                    )
                )

        print(out_tracks)
        out_container = dashbio.Igv(
            id="genome-igv", genome="hg38", locus=locus, tracks=out_tracks
        )
        return [out_container]

In [35]:
app.layout = html.Div(
    children=[
        dcc.Store(id="stored-locus"),
        html.H1(
            children="scATACseq Tracks",
            style={
                "textAlign": "center",
                "color": "#FFFFFF",
                "backgroundColor": colors["--bs-hise-blue-1"],
            },
        ),
        html.Div(
            id="dropdown_module",
            className="row",
            children=[
                html.Div(
                    className="col-3",
                    children=[
                        html.P("Select celltype(s) to view sample tracks"),
                        dcc.Dropdown(
                            id="select_celltype",
                            options=ct_order,
                            value=[ct_order[0]],
                            placeholder="Select a celltype from dropdown",
                            multi=True,
                            clearable=True,
                        ),
                    ],
                ),
                html.Div(
                    className="col-3",
                    children=[
                        html.P("Select Tracks to Display"),
                        dcc.Checklist(
                            id="show-tiles",
                            options=["Population-specific tiles",
                                     "Differential tiles", "Motif Annotations"],
                            value=["Population-specific tiles",
                                   "Differential tiles", "Motif Annotations"],
                            inline=False,
                        ),
                    ],
                ),
            ],
        ),
        html.Br(),
        html.Div(
            id="track_module",
            className="row",
            children=[
                html.P(
                    'Select a chromosome or search a specific gene or region (ex: "IKZF1" or "chr7:50,250,635-50,457,906")'
                ),
                dcc.Loading(
                    id="default-igv-container",
                    type="circle",
                    children=[
                        dashbio.Igv(
                            id="genome-igv",
                            genome="hg38",
                            locus="chr6:31,801,952-31,812,376",
                        )
                    ],
                ),
            ],
        ),
    ]
)

del app.config._read_only["requests_pathname_prefix"]
# app.run_server(debug=False, mode='inline')
app.run_server(debug=True, mode='Jupyterlab', host = '0.0.0.0')

# Look for diff peak differences chr6:31,801,952-31,812,376

Dash is running on http://0.0.0.0:8050/proxy/8050/

